# GitHub Tracking Application Part 3
Include a Repo Analysis Page
Include a checkbox for switching between line chart and statistical summary

# Setup the GitHub Credentials

In [2]:
github_user = "dtaieb"
github_token = "XXXX"

## Create the loader function for the commit activity GitHub API

In [3]:
from datetime import datetime
import requests
import pixiedust
import pandas
def load_commit_activity(owner, repo_name):
    response = requests.get(
        "https://api.github.com/repos/{}/{}/stats/commit_activity".format(owner, repo_name),
        auth=(github_user, github_token)
    ).json()
    pdf = pandas.DataFrame([
        {"total": item["total"], "week":datetime.fromtimestamp(item["week"])} for item in response
    ])
    
    return {
        "pdf":pdf,
        "chart_options": {
          "handlerId": "lineChart",
          "keyFields": "week",
          "valueFields": "total",
          "aggregation": "SUM",
          "rendererId": "bokeh"
        }
    }

## Use display to get the chart options JSON payload

In [4]:
display(load_commit_activity("ibm-watson-data-lab","pixiedust")["pdf"])

## Create the array that controls the type of data being analyzed

In [5]:
analyses = [("Commit Activity", load_commit_activity)]

## Implement the RepoAnalysis PixieApp that is responsible for visualizing the data

In [14]:
from pixiedust.display.app import *
import requests
import pandas

@PixieApp
class RepoAnalysis():
    def setup(self):
        self.show_stats = False

    @route(analyse_repo_owner="*", analyse_repo_name="*")
    @templateArgs
    def do_analyse_repo(self, analyse_repo_owner, analyse_repo_name):
        self._analyse_repo_owner = analyse_repo_owner
        self._analyse_repo_name = analyse_repo_name
        return """
<div class="container-fluid">
    <div class="col-sm-2">
        <div class="dropdown center-block">
          <button class="btn btn-primary dropdown-toggle" type="button" data-toggle="dropdown">
              Select Repo Data Set
              <span class="caret"></span>
          </button>
          <ul class="dropdown-menu" style="list-style:none;margin:0px;padding:0px">
              {%for analysis,_ in this.analyses%}
                <li>
                    <a href="#" pd_options="analyse_type={{analysis}}" pd_target="analyse_vis{{prefix}}"
                        style="text-decoration: none;background-color:transparent">
                        {{analysis}}
                    </a>
                </li>
              {%endfor%}
          </ul>
        </div>
        <div class="checkbox">
          <label><input type="checkbox" pd_script="self.show_stats=True">Show Statistics</label>
        </div>
    </div>
    <div id="analyse_vis{{prefix}}" class="col-sm-10"></div>
</div>
"""
    def get_pdf(self):
        if self.show_stats:
            summary = self.pdf.describe()
            summary.insert(0, "Stat", summary.index)
            return summary
        return self.pdf
    
    @route(analyse_type="*")
    @templateArgs
    def do_analyse_type(self, analyse_type):
        fn = [analysis_fn for a_type,analysis_fn in analyses if a_type == analyse_type]
        if len(fn) == 0:
            return "No loader function found for {{analyse_type}}"
        vis_info = fn[0](self._analyse_repo_owner, self._analyse_repo_name)
        self.pdf = vis_info["pdf"]
        chart_options = {"handlerId":"dataframe"} if self.show_stats else vis_info["chart_options"]
        return """
        <div pd_entity="get_pdf()" pd_render_onload>
            <pd_options>{{chart_options | tojson}}</pd_options>
        </div>
        """

## GitHubTracking PixieApp class
Main PixieApp class that inherits from RepoAnalysis

In [15]:
@PixieApp
class GitHubTracking(RepoAnalysis):
    @route()
    def main_screen(self):
        return """
<style>
    div.outer-wrapper {
        display: table;width:100%;height:300px;
    }
    div.inner-wrapper {
        display: table-cell;vertical-align: middle;height: 100%;width: 100%;
    }
</style>
<div class="outer-wrapper">
    <div class="inner-wrapper">
        <div class="col-sm-3"></div>
        <div class="input-group col-sm-6">
          <input id="query{{prefix}}" type="text" class="form-control" placeholder="Search projects on GitHub">
          <span class="input-group-btn">
            <button class="btn btn-default" type="button" pd_options="query=$val(query{{prefix}})">Submit Query</button>
          </span>
        </div>
    </div>
</div>   
"""
    @route(query="*")
    @templateArgs
    def do_search(self, query):
        self.first_url = "https://api.github.com/search/repositories?q={}".format(query)
        self.prev_url = None
        self.next_url = None
        self.last_url = None
        
        response = requests.get(self.first_url)
        if not response.ok:
            return "<div>An Error occurred: {{response.text}}</div>"

        total_count = response.json()['total_count']
        self.next_url = response.links.get('next', {}).get('url', None)
        self.last_url = response.links.get('last', {}).get('url', None)
        return """
<h1><center>{{total_count}} repositories were found</center></h1>
<ul class="pagination">
  <li><a href="#" pd_options="page=first_url" pd_target="body{{prefix}}">First</a></li>
  <li><a href="#" pd_options="page=prev_url" pd_target="body{{prefix}}">Prev</a></li>
  <li><a href="#" pd_options="page=next_url" pd_target="body{{prefix}}">Next</a></li>
  <li><a href="#" pd_options="page=last_url" pd_target="body{{prefix}}">Last</a></li>
</ul>
<table class="table">
    <thead>
        <tr>
            <th>Repo Name</th>
            <th>Lastname</th>
            <th>URL</th>
            <th>Stars</th>
            <th>Actions</th>
        </tr>
    </thead>
    <tbody id="body{{prefix}}">
        {{this.invoke_route(this.do_retrieve_page, page='first_url')}}
    </tbody>
</table>
"""
    @route(page="*")
    @templateArgs
    def do_retrieve_page(self, page):
        url = getattr(self, page)
        if url is None:
            return "<div>No more rows</div>"
        response = requests.get(url)
        self.prev_url = response.links.get('prev', {}).get('url', None)
        self.next_url = response.links.get('next', {}).get('url', None)
        items = response.json()['items']
        return """
{%for row in items%}
<tr>
    <td>{{row['name']}}</td>
    <td>{{row.get('owner',{}).get('login', 'N/A')}}</td>
    <td><a href="{{row['html_url']}}" target="_blank">{{row['html_url']}}</a></td>
    <td>{{row['stargazers_count']}}</td>
    <td>
        <button pd_options="analyse_repo_owner={{row["owner"]["login"]}};analyse_repo_name={{row['name']}}" 
            class="btn btn-default btn-sm" title="Analyze Repo">
            <i class="fa fa-line-chart"></i>
        </button>
    </td>
</tr>
{%endfor%}
        """

app = GitHubTracking()
app.run()

Stat,total
count,52.0
mean,12.7692307692
std,13.9910551041
min,0.0
25%,4.0
50%,10.0
75%,14.25
max,59.0
